In [ ]:
#!g1.1
# почему то все ноутбуки загрузились без выводов

In [1]:
#!g1.1
import pandas as pd

In [2]:
#!g1.1
from sklearn.datasets import fetch_20newsgroups

#загрузим обучающую выборку

newsgroups_train = fetch_20newsgroups(subset='train')

In [3]:
#!g1.1

X = newsgroups_train.data
y = newsgroups_train.target

X = pd.DataFrame(X)
X.columns = ['text']

In [6]:
#!g1.1

X.head()

,text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [5]:
#!g1.1
y

array([7, 4, 4, ..., 3, 1, 8])

In [ ]:
#!g1.1
Нужно описать соответствующий класс - для датасета 20newsGroups

хотя, можно схитрить - использовать готовую реализацию класса из Торча

In [ ]:
#!g1.1

Пайплайн:
    - Датасет
    - Модель
    

#!g1.1

## ДАТАСЕТ

#!g1.1

## Модель

In [ ]:
#!g1.1

мы возьмем готовую модель - на основе дистилберта от хаггинфейс
данная модель - англоязычная и должна хорошо подойти под наш датасет - 20newsGroups

In [ ]:
#!g1.1

#!g1.1
class DistilBertForSequenceClassification(nn.Module):
    """
    Simplified version of the same class by HuggingFace.
    See transformers/modeling_distilbert.py in the transformers repository.
    """

    def __init__(self, pretrained_model_name: str, num_classes: int = None):
        """
        Args:
            pretrained_model_name (str): HuggingFace model name.
                See transformers/modeling_auto.py
            num_classes (int): the number of class labels
                in the classification task
        """
        super().__init__()

        config = AutoConfig.from_pretrained(
            pretrained_model_name, num_labels=num_classes)

        self.distilbert = AutoModel.from_pretrained(pretrained_model_name,
                                                    config=config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, num_classes)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

    def forward(self, features, attention_mask=None, head_mask=None):
        """Compute class probabilities for the input sequence.

        Args:
            features (torch.Tensor): ids of each token,
                size ([bs, seq_length]
            attention_mask (torch.Tensor): binary tensor, used to select
                tokens which are used to compute attention scores
                in the self-attention heads, size [bs, seq_length]
            head_mask (torch.Tensor): 1.0 in head_mask indicates that
                we keep the head, size: [num_heads]
                or [num_hidden_layers x num_heads]
        Returns:
            PyTorch Tensor with predicted class probabilities
        """
        assert attention_mask is not None, "attention mask is none"
        distilbert_output = self.distilbert(input_ids=features,
                                            attention_mask=attention_mask,
                                            head_mask=head_mask)
        # we only need the hidden state here and don't need
        # transformer output, so index 0
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        # we take embeddings from the [CLS] token, so again index 0
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)

        return logits